Disponível em: https://tatoeba.org/pt-br/downloads

## Características do Dataset

## Interpretando o dataset

In [126]:
import pandas as pd
dataframe = pd.read_csv("datasets/sentences_CC0.csv", sep="\t", header=None, names=["id", "lang", "text", "created_at"])

In [127]:
dataframe.head()

,id,lang,text,created_at
0,330998,eng,Children who spend more time outdoors have a l...,2019-01-12 19:39:42
1,331000,eng,The idea that reading makes you short-sighted ...,2019-01-12 19:39:42
2,331259,eng,Most people think computers will never be able...,2019-01-12 19:39:42
3,331708,fra,Je rajoute une phrase d'exemple.,2019-01-12 19:39:42
4,331955,fra,J'espère que je ne serai plus malade la semain...,2019-01-12 19:39:42


In [128]:
print(dataframe.shape)

(556930, 4)


### Transformação, tratamento e limpeza dos dados

In [129]:
# As respectivas colunas não servem para nada nesse problema
dataframe.drop(columns=["id", "created_at"], inplace=True)

In [130]:
dataframe.head()

,lang,text
0,eng,Children who spend more time outdoors have a l...
1,eng,The idea that reading makes you short-sighted ...
2,eng,Most people think computers will never be able...
3,fra,Je rajoute une phrase d'exemple.
4,fra,J'espère que je ne serai plus malade la semain...


### Vamos ver agora a proporção de cada idioma

In [131]:
print(dataframe["lang"].value_counts(normalize=True) * 100)

lang
kab    67.017938
ber    16.537626
eng     7.445999
rus     3.948611
fra     2.538200
         ...    
zgh     0.000180
cmn     0.000180
hoc     0.000180
non     0.000180
ile     0.000180
Name: proportion, Length: 62, dtype: float64


In [132]:
# Frequência percentual de cada idioma
lang_freq = dataframe["lang"].value_counts(normalize=True) * 100

# Filtra apenas os idiomas com mais de 1%
langs_over_2 = lang_freq[lang_freq > 0.25]

print(langs_over_2)


lang
kab    67.017938
ber    16.537626
eng     7.445999
rus     3.948611
fra     2.538200
ben     1.284003
bel     0.346363
fin     0.277055
Name: proportion, dtype: float64


## Kabyle e Berber estão desproporcionando o dataset
Kabyle: Idioma falado principalmente na Argélia.

Berber: Idioma falado no norte da África (Marrocos, Argélia, Tunísia, Líbia)


### Decisão
Ao invés de apagar os dados, vou simplesmente rebalancear-los!

Levando em conta que 67% de 550 mil é 368 mil registros e 2% de 550 mil é 11 mil, vamos ajustar para a proporção mínima acima de 1% indicado no percentual acima.


In [133]:
kab_ber = dataframe[dataframe["lang"].isin(["kab", "ber"])]

# Pega a quantidade de amostrar para cada classe
amostras_kab = kab_ber[kab_ber["lang"] == "kab"].sample(n=11000, random_state=42)
amostras_ber = kab_ber[kab_ber["lang"] == "ber"].sample(n=10000, random_state=42)

# Separar o restante do dataset (não kab/ber)
rest = dataframe[~dataframe["lang"].isin(["kab", "ber"])]

# Concatenar tudo de volta
dataframe = pd.concat([rest, amostras_kab, amostras_ber])

# Embaralhar o dataframe
dataframe = dataframe.sample(frac=1, random_state=42).reset_index(drop=True)

# Conferir distribuição final
print(dataframe["lang"].value_counts())


lang
eng    41469
rus    21991
fra    14136
kab    11000
ber    10000
       ...  
non        1
yue        1
lad        1
nys        1
jpa        1
Name: count, Length: 62, dtype: int64


In [134]:
# Frequência percentual de cada idioma
lang_freq = dataframe["lang"].value_counts(normalize=True) * 100

# Filtra apenas os idiomas com mais de 1%
langs_over_2 = lang_freq[lang_freq > 1]

print(langs_over_2)

lang
eng    36.833831
rus    19.532971
fra    12.555958
kab     9.770482
ber     8.882257
ben     6.351702
bel     1.713387
fin     1.370532
Name: proportion, dtype: float64


In [135]:
dataframe.shape

(112584, 2)

# 0.5% == cerca de 4579 registros.

Feito essa limpeza de idiomas, vamos limpar as colunas irrelevantes: ID e created_at.